In [17]:
import pandas as pd
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

In [18]:
try:
    FILES
    FILE = FILES[2]
    FOLDER
except NameError:
    FILE = ['03_sell_in_lag.parquet']
    FOLDER = 'DefaultFolder'

    
print(f"------------------------------COMIENZA-----------------------------\n[{FOLDER}/{FILE}")

------------------------------COMIENZA-----------------------------
[DefaultFolder/['03_sell_in_lag.parquet']


In [19]:
# df = pd.read_parquet("sell_in_norm.parquet")
# prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')
# prod_data = pd.read_parquet('prod_data.parquet')

df = pd.read_parquet(f"{FOLDER}/{FILES[1][0]}")
prod_data = pd.read_parquet(f"{FOLDER}/{FILES[0][2]}")
prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')

# df = pd.read_parquet(f"./Experimentos/2024-06-23_NuevoScript/02_sell_in_norm.parquet")
# prod_data = pd.read_parquet('./Experimentos/2024-06-23_NuevoScript/01_prod_data_group.parquet')
# prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')

## Pendientes
---
### operaciones sobre tn
* HECHO - razones con lags
* HECHO - marcar cero real
* HECHO - date features:
    - HECHO - mes
    - HECHO - quarter
    - HECHO - mes en el quarter
* HECHO - tiempos de monotonia de la funcion
* maximo salto
* expanding mean

### basadas en producto
* HECHO - info del producto
* HECHO porcentual de venta
    - HECHO del mes
    - HECHO del quarter
    - HECHO del año
* porcentual de categoria
* HECHO - stats de categoria1 y 2
* HECHO - stats de brand

## Generales

In [20]:
# date features
df['periodo_dt'] = pd.to_datetime(df['periodo'], format='%Y%m')
df = df.sort_values(by=['group_index', 'periodo_dt'])
df['mes'] = df['periodo_dt'].dt.month
df['quarter'] = df['periodo_dt'].dt.quarter
df['month_in_quarter'] = (df['periodo_dt'].dt.month - 1) % 3 + 1
df['year'] = df['periodo_dt'].dt.year
df['yearquarter'] = df['periodo_dt'].dt.to_period('Q').astype('str').astype('category')

print(df[df['periodo'] == 201912].shape)

(542, 20)


## Operaciones sobre tn

In [21]:
df

,group_index,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,median_tn,std_dev_tn,iqr_tn,max_tn,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,yearquarter
0,0,201701,6.32131,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,2.164849,3.344115,9.23561,0.166328,2017-01-01,1,1,1,2017,2017Q1
1,0,201702,5.36212,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,2.164849,3.344115,9.23561,-0.276747,2017-02-01,2,1,2,2017,2017Q1
2,0,201703,4.27708,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,2.164849,3.344115,9.23561,-0.777955,2017-03-01,3,1,3,2017,2017Q1
3,0,201704,7.85708,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,2.164849,3.344115,9.23561,0.875740,2017-04-01,4,2,1,2017,2017Q2
4,0,201705,5.64517,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,2.164849,3.344115,9.23561,-0.145999,2017-05-01,5,2,2,2017,2017Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,836,201908,0.04442,2017-01-01,2019-12-01,36.0,3.91042,0.02184,0.108623,0.081180,0.111176,0.053700,0.53294,-0.577488,2019-08-01,8,3,2,2019,2019Q3
15412,836,201909,0.08373,2017-01-01,2019-12-01,36.0,3.91042,0.02184,0.108623,0.081180,0.111176,0.053700,0.53294,-0.223904,2019-09-01,9,3,3,2019,2019Q3
15413,836,201910,0.06699,2017-01-01,2019-12-01,36.0,3.91042,0.02184,0.108623,0.081180,0.111176,0.053700,0.53294,-0.374476,2019-10-01,10,4,1,2019,2019Q4
15414,836,201911,0.02766,2017-01-01,2019-12-01,36.0,3.91042,0.02184,0.108623,0.081180,0.111176,0.053700,0.53294,-0.728240,2019-11-01,11,4,2,2019,2019Q4


In [22]:
# marcar ceros
df['tn_cero'] = df['tn'] < df['median_tn'] / 10

In [23]:
lags = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15,18,21,24,30,36]
lags

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 18, 21, 24, 30, 36]

In [24]:
# lag features
for lag in lags:
    # rolling max
    df[f'max_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).max() == df['tn_norm']
    # rolling min
    df[f'min_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).min() == df['tn_norm']
    # rolling avg
    df[f'avg_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).mean()
    # lags
    df[f'tn_lag_{lag}'] = df.groupby('group_index')['tn_norm'].shift(lag)
    # primer derivada / delta
    df[f'tn_diff_{lag}'] = df['tn_norm'].diff(lag)
    # ratios con lags
    df[f'ratio_{lag}'] = (-df[f'tn_diff_{lag}'])/df[f'tn_lag_{lag}']

for deriv in range(1, 13):
    # segunda derivada
    df[f'tn_diff2_{deriv}'] = df[f'tn_diff_{deriv}'].diff()

In [25]:
# monotonia

for i in lags:
    df[f'crece_{i+1}'] = (df.groupby('group_index')['tn_norm'].shift(i) - df.groupby('group_index')['tn_norm'].shift(i+1)) > 0

crece_columns = df.filter(like='crece_')

df['crece_sum'] = crece_columns.sum(axis=1)
df['decrece_sum'] = 12 - crece_columns.sum(axis=1) #corregir para los primeros

## nivel producto

In [26]:
df = pd.merge(df, prod_data, on='group_index', how='left', suffixes=('', ''))

In [27]:
print(df[df['periodo'] == 201912].shape)

(542, 168)


In [28]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [29]:
variables = ['cat1', 'cat2', 'brand']
var_stats = []

for variable in variables:
    var_stat = df.groupby(variable).apply(lambda group: pd.Series({
        f'{variable}_total': group['tn'].sum(),
        #f'{variable}_min': group['tn'].min(),
        #f'{variable}_average': group['tn'].mean(),
        f'{variable}_median': group['tn'].median(),
        #f'{variable}_std_dev': group['tn'].std(),
        #f'{variable}_iqr': (group['tn'].quantile(0.75) - group['tn'].quantile(0.25)),
        f'{variable}_max': group['tn'].max()
    }))
    var_stat.reset_index(inplace=True)
    var_stats.append(var_stat)

# Merge yearly sales with the products dataframe
#df_prod = df_prod.merge(df_temp, on='product_id', how='left')

In [30]:
for i in range(len(variables)):
    var_stat = var_stats[i]
    df = pd.merge(df, var_stat, on=variables[i], how='left')

In [31]:
tiempos = ['periodo','yearquarter','year']

for tiempo in tiempos:
    tiempo_sums = df.groupby(tiempo)['tn'].sum().reset_index()
    tiempo_sums.rename(columns={'tn': f'tn_total_{tiempo}'}, inplace=True)
    df = df.merge(tiempo_sums, on=tiempo)
    df[f'prop_product_{tiempo}'] = (df['tn'] / df[f'tn_total_{tiempo}']) * 100
    df.drop(columns=f'tn_total_{tiempo}', inplace=True)

In [32]:
variables = ['cat1','cat2','brand']

for tiempo in tiempos:
    for variable in variables:
        combi_sums = df.groupby([tiempo, variable])['tn'].sum().reset_index()
        combi_sums = df.groupby([tiempo, variable])['tn'].sum().reset_index()
        combi_sums.rename(columns={'tn': f'tn_total_{tiempo}_{variable}'}, inplace=True)
        df = df.merge(combi_sums, on=[tiempo, variable])
        df[f'prop_product_{tiempo}_{variable}'] = (df['tn'] / df[f'tn_total_{tiempo}_{variable}']) * 100
        df.drop(columns=f'tn_total_{tiempo}_{variable}', inplace=True)

## ajustes finales pre export

In [34]:
df = df.sort_values(by=['group_index', 'periodo'])

In [35]:
df.head(15)

,group_index,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,median_tn,...,prop_product_year,prop_product_periodo_cat1,prop_product_periodo_cat2,prop_product_periodo_brand,prop_product_yearquarter_cat1,prop_product_yearquarter_cat2,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand
0,0,201701,6.32131,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.001578,0.112270,0.139281,0.169533,0.034983,0.049545,0.061778,0.007669,0.010199,0.012922
8,0,201702,5.36212,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.001338,0.088959,0.134723,0.167924,0.029675,0.042027,0.052404,0.006506,0.008652,0.010961
16,0,201703,4.27708,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.001067,0.066709,0.100873,0.129201,0.023670,0.033523,0.041800,0.005189,0.006901,0.008743
24,0,201704,7.85708,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.001961,0.131068,0.203383,0.266411,0.041798,0.065838,0.086642,0.009533,0.012677,0.016061
32,0,201705,5.64517,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.001409,0.087633,0.140221,0.186614,0.030031,0.047303,0.062250,0.006849,0.009108,0.011540
40,0,201706,9.23561,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.002305,0.145184,0.228326,0.298482,0.049131,0.077389,0.101843,0.011205,0.014902,0.018879
48,0,201707,6.15353,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.001536,0.108309,0.150812,0.193391,0.027754,0.034836,0.043989,0.007466,0.009929,0.012579
56,0,201708,8.20828,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.002049,0.114154,0.144067,0.182140,0.037021,0.046468,0.058678,0.009959,0.013244,0.016779
64,0,201709,9.00496,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.002247,0.096829,0.114180,0.142932,0.040614,0.050978,0.064373,0.010925,0.014529,0.018408
72,0,201710,8.41269,2017-01-01,2019-12-01,36.0,214.60446,0.47173,5.961235,6.234785,...,0.002100,0.086232,0.100408,0.124564,0.035976,0.042878,0.053826,0.010207,0.013574,0.017197


In [ ]:
#df.to_parquet('sell_in_lag.parquet', index=False)
df.to_parquet(f'{FOLDER}/{FILE[0]}', index=False)

In [ ]:
#estado_control = f"03_features Terminado - {nombrefile} - {datetime.now()}"

Notebook 1 terminado


In [ ]:
print(f" {FOLDER}/{FILE}\n------------------------------FINALIZA-----------------------------")

 DefaultFolder/['01_sell_in_group.parquet', '01_prod_stats.parquet', '01_prod_data.parquet', '01_stock_data.parquet']
------------------------------FINALIZA-----------------------------
